In [1]:
import os
import json
import pandas as pd
from pathlib import Path
import numpy as np
import ast
from tqdm import tqdm

# Data Extraction

In [5]:
ASJC_MAP = {
    # Multidisciplinary
    '1000': 'Multidisciplinary',

    # Agricultural and Biological Sciences
    '1100': 'General Agricultural and Biological Sciences',
    '1101': 'Agricultural and Biological Sciences (miscellaneous)',
    '1102': 'Agronomy and Crop Science',
    '1103': 'Animal Science and Zoology',
    '1104': 'Aquatic Science',
    '1105': 'Ecology, Evolution, Behavior and Systematics',
    '1106': 'Food Science',
    '1107': 'Forestry',
    '1108': 'Horticulture',
    '1109': 'Insect Science',
    '1110': 'Plant Science',
    '1111': 'Soil Science',

    # Arts and Humanities
    '1200': 'General Arts and Humanities',
    '1201': 'Arts and Humanities (miscellaneous)',
    '1202': 'History',
    '1203': 'Language and Linguistics',
    '1204': 'Archaeology',
    '1205': 'Classics',
    '1206': 'Conservation',
    '1207': 'History and Philosophy of Science',
    '1208': 'Literature and Literary Theory',
    '1209': 'Museology',
    '1210': 'Music',
    '1211': 'Philosophy',
    '1212': 'Religious Studies',
    '1213': 'Visual Arts and Performing Arts',

    # Biochemistry, Genetics and Molecular Biology
    '1300': 'General Biochemistry, Genetics and Molecular Biology',
    '1301': 'Biochemistry, Genetics and Molecular Biology (miscellaneous)',
    '1302': 'Ageing',
    '1303': 'Biochemistry',
    '1304': 'Biophysics',
    '1305': 'Biotechnology',
    '1306': 'Cancer Research',
    '1307': 'Cell Biology',
    '1308': 'Clinical Biochemistry',
    '1309': 'Developmental Biology',
    '1310': 'Endocrinology',
    '1311': 'Genetics',
    '1312': 'Molecular Biology',
    '1313': 'Molecular Medicine',
    '1314': 'Physiology',
    '1315': 'Structural Biology',

    # Business, Management and Accounting
    '1400': 'General Business, Management and Accounting',
    '1401': 'Business, Management and Accounting (miscellaneous)',
    '1402': 'Accounting',
    '1403': 'Business and International Management',
    '1404': 'Management Information Systems',
    '1405': 'Management of Technology and Innovation',
    '1406': 'Marketing',
    '1407': 'Organizational Behavior and Human Resource Management',
    '1408': 'Strategy and Management',
    '1409': 'Tourism, Leisure and Hospitality Management',
    '1410': 'Industrial Relations',

    # Chemical Engineering
    '1500': 'General Chemical Engineering',
    '1501': 'Chemical Engineering (miscellaneous)',
    '1502': 'Bioengineering',
    '1503': 'Catalysis',
    '1504': 'Chemical Health and Safety',
    '1505': 'Colloid and Surface Chemistry',
    '1506': 'Filtration and Separation',
    '1507': 'Fluid Flow and Transfer Processes',
    '1508': 'Process Chemistry and Technology',

    # Chemistry
    '1600': 'General Chemistry',
    '1601': 'Chemistry (miscellaneous)',
    '1602': 'Analytical Chemistry',
    '1603': 'Electrochemistry',
    '1604': 'Inorganic Chemistry',
    '1605': 'Organic Chemistry',
    '1606': 'Physical and Theoretical Chemistry',
    '1607': 'Spectroscopy',

    # Computer Science
    '1700': 'General Computer Science',
    '1701': 'Computer Science (miscellaneous)',
    '1702': 'Artificial Intelligence',
    '1703': 'Computational Theory and Mathematics',
    '1704': 'Computer Graphics and Computer-Aided Design',
    '1705': 'Computer Networks and Communications',
    '1706': 'Computer Science Applications',
    '1707': 'Computer Vision and Pattern Recognition',
    '1708': 'Hardware and Architecture',
    '1709': 'Human-Computer Interaction',
    '1710': 'Information Systems',
    '1711': 'Signal Processing',
    '1712': 'Software',

    # Decision Sciences
    '1800': 'General Decision Sciences',
    '1801': 'Decision Sciences (miscellaneous)',
    '1802': 'Information Systems and Management',
    '1803': 'Management Science and Operations Research',
    '1804': 'Statistics, Probability and Uncertainty',

    # Earth and Planetary Sciences
    '1900': 'General Earth and Planetary Sciences',
    '1901': 'Earth and Planetary Sciences (miscellaneous)',
    '1902': 'Atmospheric Science',
    '1903': 'Computers in Earth Sciences',
    '1904': 'Earth-Surface Processes',
    '1905': 'Economic Geology',
    '1906': 'Geochemistry and Petrology',
    '1907': 'Geology',
    '1908': 'Geophysics',
    '1909': 'Geotechnical Engineering and Engineering Geology',
    '1910': 'Oceanography',
    '1911': 'Paleontology',
    '1912': 'Space and Planetary Science',
    '1913': 'Stratigraphy',

    # Economics, Econometrics and Finance
    '2000': 'General Economics, Econometrics and Finance',
    '2001': 'Economics, Econometrics and Finance (miscellaneous)',
    '2002': 'Economics and Econometrics',
    '2003': 'Finance',

    # Energy
    '2100': 'General Energy',
    '2101': 'Energy (miscellaneous)',
    '2102': 'Energy Engineering and Power Technology',
    '2103': 'Fuel Technology',
    '2104': 'Nuclear Energy and Engineering',
    '2105': 'Renewable Energy, Sustainability and the Environment',

    # Engineering
    '2200': 'General Engineering',
    '2201': 'Engineering (miscellaneous)',
    '2202': 'Aerospace Engineering',
    '2203': 'Automotive Engineering',
    '2204': 'Biomedical Engineering',
    '2205': 'Civil and Structural Engineering',
    '2206': 'Computational Mechanics',
    '2207': 'Control and Systems Engineering',
    '2208': 'Electrical and Electronic Engineering',
    '2209': 'Industrial and Manufacturing Engineering',
    '2210': 'Mechanical Engineering',
    '2211': 'Mechanics of Materials',
    '2212': 'Ocean Engineering',
    '2213': 'Safety, Risk, Reliability and Quality',
    '2214': 'Media Technology',
    '2215': 'Building and Construction',
    '2216': 'Architecture',

    # Environmental Science
    '2300': 'General Environmental Science',
    '2301': 'Environmental Science (miscellaneous)',
    '2302': 'Ecological Modeling',
    '2303': 'Ecology',
    '2304': 'Environmental Chemistry',
    '2305': 'Environmental Engineering',
    '2306': 'Global and Planetary Change',
    '2307': 'Health, Toxicology and Mutagenesis',
    '2308': 'Management, Monitoring, Policy and Law',
    '2309': 'Nature and Landscape Conservation',
    '2310': 'Pollution',
    '2311': 'Waste Management and Disposal',
    '2312': 'Water Science and Technology',

    # Immunology and Microbiology
    '2400': 'General Immunology and Microbiology',
    '2401': 'Immunology and Microbiology (miscellaneous)',
    '2402': 'Applied Microbiology and Biotechnology',
    '2403': 'Immunology',
    '2404': 'Microbiology',
    '2405': 'Parasitology',
    '2406': 'Virology',

    # Materials Science
    '2500': 'General Materials Science',
    '2501': 'Materials Science (miscellaneous)',
    '2502': 'Biomaterials',
    '2503': 'Ceramics and Composites',
    '2504': 'Electronic, Optical and Magnetic Materials',
    '2505': 'Materials Chemistry',
    '2506': 'Metals and Alloys',
    '2507': 'Polymers and Plastics',
    '2508': 'Surfaces, Coatings and Films',

    # Mathematics
    '2600': 'General Mathematics',
    '2601': 'Mathematics (miscellaneous)',
    '2602': 'Algebra and Number Theory',
    '2603': 'Analysis',
    '2604': 'Applied Mathematics',
    '2605': 'Computational Mathematics',
    '2606': 'Control and Optimization',
    '2607': 'Discrete Mathematics and Combinatorics',
    '2608': 'Geometry and Topology',
    '2609': 'Logic',
    '2610': 'Mathematical Physics',
    '2611': 'Modeling and Simulation',
    '2612': 'Numerical Analysis',
    '2613': 'Statistics and Probability',
    '2614': 'Theoretical Computer Science',

    # Medicine
    '2700': 'General Medicine',
    '2701': 'Medicine (miscellaneous)',
    '2702': 'Anatomy',
    '2703': 'Anesthesiology and Pain Medicine',
    '2704': 'Biochemistry (medical)',
    '2705': 'Cardiology and Cardiovascular Medicine',
    '2706': 'Critical Care and Intensive Care Medicine',
    '2707': 'Complementary and Alternative Medicine',
    '2708': 'Dermatology',
    '2709': 'Drug Guides',
    '2710': 'Embryology',
    '2711': 'Emergency Medicine',
    '2712': 'Endocrinology, Diabetes and Metabolism',
    '2713': 'Epidemiology',
    '2714': 'Family Practice',
    '2715': 'Gastroenterology',
    '2716': 'Genetics (clinical)',
    '2717': 'Geriatrics and Gerontology',
    '2718': 'Health Informatics',
    '2719': 'Health Policy',
    '2720': 'Hematology',
    '2721': 'Hepatology',
    '2722': 'Histology',
    '2723': 'Immunology and Allergy',
    '2724': 'Internal Medicine',
    '2725': 'Infectious Diseases',
    '2726': 'Microbiology (medical)',
    '2727': 'Nephrology',
    '2728': 'Neurology (clinical)',
    '2729': 'Obstetrics and Gynecology',
    '2730': 'Oncology',
    '2731': 'Ophthalmology',
    '2732': 'Orthopedics and Sports Medicine',
    '2733': 'Otorhinolaryngology',
    '2734': 'Pathology and Forensic Medicine',
    '2735': 'Pediatrics, Perinatology and Child Health',
    '2736': 'Pharmacology (medical)',
    '2737': 'Physiology (medical)',
    '2738': 'Psychiatry and Mental Health',
    '2739': 'Public Health, Environmental and Occupational Health',
    '2740': 'Pulmonary and Respiratory Medicine',
    '2741': 'Radiology, Nuclear Medicine and Imaging',
    '2742': 'Rehabilitation',
    '2743': 'Reproductive Medicine',
    '2744': 'Reviews and References (medical)',
    '2745': 'Rheumatology',
    '2746': 'Surgery',
    '2747': 'Transplantation',
    '2748': 'Urology',

    # Neuroscience
    '2800': 'General Neuroscience',
    '2801': 'Neuroscience (miscellaneous)',
    '2802': 'Behavioral Neuroscience',
    '2803': 'Biological Psychiatry',
    '2804': 'Cellular and Molecular Neuroscience',
    '2805': 'Cognitive Neuroscience',
    '2806': 'Developmental Neuroscience',
    '2807': 'Endocrine and Autonomic Systems',
    '2808': 'Neurology',
    '2809': 'Sensory Systems',

    # Nursing
    '2900': 'General Nursing',
    '2901': 'Nursing (miscellaneous)',
    '2902': 'Advanced and Specialized Nursing',
    '2903': 'Assessment and Diagnosis',
    '2904': 'Care Planning',
    '2905': 'Community and Home Care',
    '2906': 'Critical Care Nursing',
    '2907': 'Emergency Nursing',
    '2908': 'Fundamentals and Skills',
    '2909': 'Gerontology',
    '2910': 'Issues, Ethics and Legal Aspects',
    '2911': 'Leadership and Management',
    '2912': 'LPN and LVN',
    '2913': 'Maternity and Midwifery',
    '2914': 'Medical and Surgical Nursing',
    '2915': 'Nurse Assisting',
    '2916': 'Nutrition and Dietetics',
    '2917': 'Oncology (nursing)',
    '2918': 'Pathophysiology',
    '2919': 'Pediatrics',
    '2920': 'Pharmacology (nursing)',
    '2921': 'Psychiatric and Mental Health',
    '2922': 'Research and Theory',
    '2923': 'Review and Exam Preparation',

    # Pharmacology, Toxicology and Pharmaceutics
    '3000': 'General Pharmacology, Toxicology and Pharmaceutics',
    '3001': 'Pharmacology, Toxicology and Pharmaceutics (miscellaneous)',
    '3002': 'Drug Discovery',
    '3003': 'Pharmaceutical Science',
    '3004': 'Pharmacology',
    '3005': 'Toxicology',

    # Physics and Astronomy
    '3100': 'General Physics and Astronomy',
    '3101': 'Physics and Astronomy (miscellaneous)',
    '3102': 'Acoustics and Ultrasonics',
    '3103': 'Astronomy and Astrophysics',
    '3104': 'Condensed Matter Physics',
    '3105': 'Instrumentation',
    '3106': 'Nuclear and High Energy Physics',
    '3107': 'Atomic and Molecular Physics, and Optics',
    '3108': 'Radiation',
    '3109': 'Statistical and Nonlinear Physics',
    '3110': 'Surfaces and Interfaces',

    # Psychology
    '3200': 'General Psychology',
    '3201': 'Psychology (miscellaneous)',
    '3202': 'Applied Psychology',
    '3203': 'Clinical Psychology',
    '3204': 'Developmental and Educational Psychology',
    '3205': 'Experimental and Cognitive Psychology',
    '3206': 'Neuropsychology and Physiological Psychology',
    '3207': 'Social Psychology',

    # Social Sciences
    '3300': 'General Social Sciences',
    '3301': 'Social Sciences (miscellaneous)',
    '3302': 'Archaeology',
    '3303': 'Development',
    '3304': 'Education',
    '3305': 'Geography, Planning and Development',
    '3306': 'Health (social science)',
    '3307': 'Human Factors and Ergonomics',
    '3308': 'Law',
    '3309': 'Library and Information Sciences',
    '3310': 'Linguistics and Language',
    '3311': 'Safety Research',
    '3312': 'Sociology and Political Science',
    '3313': 'Transportation',
    '3314': 'Anthropology',
    '3315': 'Communication',
    '3316': 'Cultural Studies',
    '3317': 'Demography',
    '3318': 'Gender Studies',
    '3319': 'Life-span and Life-course Studies',
    '3320': 'Political Science and International Relations',
    '3321': 'Public Administration',
    '3322': 'Urban Studies',

    # Veterinary
    '3400': 'General Veterinary',
    '3401': 'Veterinary (miscellaneous)',
    '3402': 'Equine',
    '3403': 'Food Animals',
    '3404': 'Small Animals',

    # Dentistry
    '3500': 'General Dentistry',
    '3501': 'Dentistry (miscellaneous)',
    '3502': 'Dental Assisting',
    '3503': 'Dental Hygiene',
    '3504': 'Oral Surgery',
    '3505': 'Orthodontics',
    '3506': 'Periodontics',

    # Health Professions
    '3600': 'General Health Professions',
    '3601': 'Health Professions (miscellaneous)',
    '3602': 'Chiropractics',
    '3603': 'Complementary and Manual Therapy',
    '3604': 'Emergency Medical Services',
    '3605': 'Health Information Management',
    '3606': 'Medical Assisting and Transcription',
    '3607': 'Medical Laboratory Technology',
    '3608': 'Medical Terminology',
    '3609': 'Occupational Therapy',
    '3610': 'Optometry',
    '3611': 'Pharmacy',
    '3612': 'Physical Therapy, Sports Therapy and Rehabilitation',
    '3613': 'Podiatry',
    '3614': 'Radiological and Ultrasound Technology',
    '3615': 'Respiratory Care',
    '3616': 'Speech and Hearing',
}

def get_nested(data, path, default=None):
    """Safely navigates nested dictionaries."""
    try:
        for key in path:
            data = data[key]
        return data
    except (KeyError, TypeError, AttributeError):
        return default

def get_scopus_id(itemidlist):
    try:
        if not itemidlist or 'itemid' not in itemidlist: return None
        for item in itemidlist.get('itemid', []):
            if item.get('@idtype') == 'SCP': return item.get('$')
    except: return None

def get_authors(author_groups):
    try:
        if not author_groups: return None
        if isinstance(author_groups, dict): author_groups = [author_groups]
        authors = {}
        for group in author_groups:
            alist = group.get('author', [])
            if isinstance(alist, dict): alist = [alist]
            for a in alist:
                if a.get('@auid') and a.get('preferred-name', {}).get('ce:indexed-name'):
                    authors[a['@auid']] = a['preferred-name']['ce:indexed-name']
        return "; ".join(authors.values())
    except: return None

def get_affiliations(author_groups):
    try:
        if not author_groups: return None
        if isinstance(author_groups, dict): author_groups = [author_groups]
        affil_set = set()
        for group in author_groups:
            affil = group.get('affiliation', {})
            org = affil.get('organization', {})
            if isinstance(org, list):
                name = ", ".join([o.get('$') for o in org if o.get('$')])
            elif isinstance(org, dict):
                name = org.get('$')
            else: name = None
            if name: affil_set.add(name)
        return "; ".join(affil_set)
    except: return None

def get_asjc_raw(classificationgroup):
    try:
        if not classificationgroup or 'classifications' not in classificationgroup: return np.nan
        clist = classificationgroup.get('classifications', [])
        if isinstance(clist, dict): clist = [clist]
        codes = [c.get('classification') for c in clist if c.get('@type') == 'ASJC']
        if not codes: return np.nan
        if len(codes) == 1: return codes[0]
        return str([{"$": c} for c in codes]) 
    except: return np.nan

def translate_asjc(raw_val, mapper):
    if np.all(pd.isna(raw_val)): return np.nan
    s = str(raw_val).strip()
    try:
        if s.startswith('['):
            data = ast.literal_eval(s)
            codes = [i['$'] for i in data if isinstance(i, dict) and '$' in i]
        else:
            codes = [s]
        return ", ".join([mapper.get(c, f"Unknown({c})") for c in codes])
    except: return "Error"

def get_abstract(head):
    try:
        data = head.get('abstracts')
        if isinstance(data, str): return data
        if isinstance(data, dict): return data.get('$', None)
    except: return None

def get_cover_date(source, coredata, process_info):
    """Tries multiple places to find a valid date."""
    try:
        
        if coredata and coredata.get('prism:coverDate'):
            return coredata.get('prism:coverDate')

        
        pd = source.get('publicationdate', {})
        if pd.get('year') and pd.get('month') and pd.get('day'):
            return f"{pd['year']}-{pd['month']}-{pd['day']}"
        
        
        sd = process_info.get('ait:date-sort', {})
        if sd.get('@year') and sd.get('@month') and sd.get('@day'):
            return f"{sd['@year']}-{sd['@month']}-{sd['@day']}"
        
        
        if pd.get('year'):
            return f"{pd['year']}-{pd.get('month', '01')}-01"
            
        return None
    except: return None

def get_author_keywords(root_data):
    """Extracts keywords from the root authkeywords block."""
    try:
        keywords = root_data.get('authkeywords', {}).get('author-keyword', [])
        if isinstance(keywords, dict): keywords = [keywords]
        return "; ".join([k.get('$', '') for k in keywords if '$' in k])
    except: return None

def process_folder_full_columns(root_path, output_csv):
    all_data = []
    files_processed = 0
    
    print(f"Scanning all files in: {root_path}")

    if not os.path.exists(root_path):
        print("Error: Folder path does not exist.")
        return

    for root, dirs, files in os.walk(root_path):
        for fname in tqdm(files, desc=f"Reading {os.path.basename(root)}"):
            if fname.startswith('.') or fname.endswith('.csv'): continue
            
            fpath = os.path.join(root, fname)
            
            try:
                with open(fpath, 'r', encoding='utf-8') as f:
                    content = f.read()
                
                data = json.loads(content)
                root_response = data.get('abstracts-retrieval-response', {})
                item = root_response.get('item', {})
                coredata = root_response.get('coredata', {}) 
                
                if not item: continue
                
                bib = item.get('bibrecord', {})
                head = bib.get('head', {})
                source = head.get('source', {})
                info = bib.get('item-info', {})
                
                ct = head.get('citation-title')
                chapter_title = ct.get('titletext') if isinstance(ct, dict) else ct
                
                doi = info.get('itemidlist', {}).get('ce:doi')
                scopus_id = get_scopus_id(info.get('itemidlist', {}))
                
                pub_year = source.get('publicationyear', {}).get('@first')
                book_title = source.get('sourcetitle')
                publisher = source.get('publisher', {}).get('publishername')
                
                authors = get_authors(head.get('author-group', []))
                affiliation = get_affiliations(head.get('author-group', []))
                
                asjc_raw = get_asjc_raw(head.get('enhancement', {}).get('classificationgroup'))
                asjc_trans = translate_asjc(asjc_raw, ASJC_MAP)
                
                abstract = get_abstract(head)
                cover_date = get_cover_date(source, coredata, item.get('ait:process-info', {}))
                description = coredata.get('dc:description')
                agg_type = coredata.get('prism:aggregationType')
                auth_keywords = get_author_keywords(root_response)
                ref_count = bib.get('tail', {}).get('bibliography', {}).get('@refcount')
                row = {
                    "file_name": fname,
                    "chapter_title": chapter_title,
                    "doi": doi,
                    "scopus_id": scopus_id,
                    "publication_year": pub_year,
                    "book_title": book_title,
                    "publisher": publisher,
                    "authors": authors,
                    "affiliation": affiliation,
                    "ASJC": asjc_raw,
                    "ASJC_translation": asjc_trans,
                    "abstract": abstract,
                    "cover_date": cover_date,        
                    "description": description,      
                    "aggregation_type": agg_type,    
                    "author_keywords": auth_keywords,
                    "reference_count": ref_count     
                }
                all_data.append(row)
                files_processed += 1

            except json.JSONDecodeError:
                pass 
            except Exception:
                pass 
    
    if all_data:
        df = pd.DataFrame(all_data)
        cols = [
            "file_name", "chapter_title", "doi", "scopus_id", 
            "publication_year", "cover_date", 
            "book_title", "publisher", "aggregation_type",
            "authors", "affiliation", 
            "abstract", "description", "author_keywords",
            "ASJC", "ASJC_translation", "reference_count"
        ]
        
        cols = [c for c in cols if c in df.columns]
        df = df[cols]
        
        df.to_csv(output_csv, index=False)
        print(f"\nDone! Processed {files_processed} valid papers.")
        print(f"Data saved to: {output_csv}")
        print(df.head(2))
    else:
        print("\nNo valid Scopus JSON data found.")

target_folder = "C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/"
output_filename = "extracted_data.csv"

process_folder_full_columns(target_folder, output_filename)

Scanning all files in: C:/work/Year3/data sci/Project-Data-Sci/ScopusData2018-2023/


Reading 2023: 100%|██████████| 2890/2890 [01:17<00:00, 37.09it/s]



Done! Processed 19805 valid papers.
Data saved to: extracted_data.csv
   file_name                                      chapter_title  \
0  201800000  Public health and international epidemiology f...   
1  201800001  Flexible Printed Active Antenna for Digital Te...   

                            doi    scopus_id publication_year  cover_date  \
0  10.1007/978-3-319-98485-8_15  85077976956             2018  2018-12-31   
1   10.23919/PIERS.2018.8597669  85060936020             2018  2018-12-31   

                                          book_title  \
0  Radiology in Global Health: Strategies, Implem...   
1    Progress in Electromagnetics Research Symposium   

                                           publisher       aggregation_type  \
0                  Springer International Publishing                   Book   
1  Institute of Electrical and Electronics Engine...  Conference Proceeding   

                       authors  \
0   Pongpirul K.; Lungren M.P.   
1  Pratumsiri T.; Ja

In [6]:
df = pd.read_csv("extracted_data.csv")
df

,file_name,chapter_title,doi,scopus_id,publication_year,cover_date,book_title,publisher,aggregation_type,authors,affiliation,abstract,description,author_keywords,ASJC,ASJC_translation,reference_count
0,201800000,Public health and international epidemiology f...,10.1007/978-3-319-98485-8_15,85077976956,2018,2018-12-31,"Radiology in Global Health: Strategies, Implem...",Springer International Publishing,Book,Pongpirul K.; Lungren M.P.,"Department of Preventive and Social Medicine, ...",NaN,NaN,NaN,2700,General Medicine,76
1,201800001,Flexible Printed Active Antenna for Digital Te...,10.23919/PIERS.2018.8597669,85060936020,2018,2018-12-31,Progress in Electromagnetics Research Symposium,Institute of Electrical and Electronics Engine...,Conference Proceeding,Pratumsiri T.; Janpugdee P.,"Department of Electrical Engineering, Wireless...","© 2018 The Institute of Electronics, Informati...",This paper presents the development of a flexi...,NaN,"[{'$': '2208'}, {'$': '2504'}]","Electrical and Electronic Engineering, Electro...",4
2,201800002,Parametric study of hydrogen production via so...,10.1016/j.ces.2018.08.042,85052201238,2018,2018-12-31,Chemical Engineering Science,Elsevier Ltd,Journal,Phuakpunk K.; Assabumrungrat S.; Chalermsinsuw...,Advanced Computational Fluid Dynamics Research...,© 2018 Elsevier LtdComputational fluid dynamic...,Computational fluid dynamics was applied for s...,Circulating fluidized bed; Computational fluid...,"[{'$': '1600'}, {'$': '1500'}, {'$': '2209'}]","General Chemistry, General Chemical Engineerin...",42
3,201800003,Superhydrophobic coating from fluoroalkylsilan...,10.1016/j.apsusc.2018.08.059,85051498032,2018,2018-12-31,Applied Surface Science,Elsevier B.V.,Journal,Saengkaew J.; Le D.; Samart C.; Kongparakul S....,"Faculty of Science, Chulalongkorn University; ...",© 2018 Elsevier B.V. A superhydrophobic/supero...,A superhydrophobic/superoleophilic mesh was su...,Encapsulation; Fluoroalkylsilane; Natural rubb...,"[{'$': '1600'}, {'$': '3104'}, {'$': '3100'}, ...","General Chemistry, Condensed Matter Physics, G...",45
4,201800004,Electrochemical impedance-based DNA sensor usi...,10.1016/j.aca.2018.07.045,85050678366,2018,2018-12-31,Analytica Chimica Acta,Elsevier B.V.,Journal,Teengam P.; Siangproh W.; Tuantranont A.; Vila...,"Organic Synthesis Research Unit, Department of...",© 2018 Elsevier B.V. A label-free electrochemi...,A label-free electrochemical DNA sensor based ...,acpcPNA; Electrochemical impedance spectroscop...,"[{'$': '1602'}, {'$': '1303'}, {'$': '2304'}, ...","Analytical Chemistry, Biochemistry, Environmen...",55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19800,202302885,Long-chain bio-olefins production via oxidativ...,10.1016/j.cattod.2021.07.034,85111945558,2023,2023-01-01,Catalysis Today,Elsevier B.V.,Journal,Le D.; Hinchiranan N.; Chaidherasuwet N.; Ruea...,Center of Excellence on Petrochemical and Mate...,© 2021 Elsevier B.V.Long-chain α-olefins (≥ C1...,Long-chain α-olefins (≥ C10) are normally appl...,Long-chain olefins; Mesoporous KIT-6; Oleic ac...,"[{'$': '1503'}, {'$': '1600'}]","Catalysis, General Chemistry",63
19801,202302886,Recent Developments and Applications of Microf...,10.1080/10408347.2021.1949695,85111408415,2023,2023-01-01,Critical Reviews in Analytical Chemistry,Taylor and Francis Ltd.,Journal,Alahmad W.; Varanusupakul P.; Varanusupakul P.,"Department of Chemistry, Faculty of Science, C...","© 2021 Taylor & Francis Group, LLC.Nowadays, f...","Nowadays, food safety has become a major conce...",Biological hazards; chemical hazards; food con...,1602,Analytical Chemistry,115
19802,202302887,"Social justice, education and peacebuilding: c...",10.1080/03057925.2021.1951666,85110903700,2023,2023-01-01,Compare,Routledge,Journal,Pherali T.,Centre for Education and International Develop...,© 2021 The Author(s). Published by Informa UK ...,Education is increasingly becoming central to ...,conflict; Education; peacebuilding; social jus...,3304,Ed

# Data Preprocessing

In [40]:
cols = [
    "chapter_title",
    "abstract",
    "author_keywords",
    "description",
    "book_title",
    "publication_year",
    "ASJC_translation",
]

df_clean = df[cols].copy()

text_cols = [
    "chapter_title",
    "abstract",
    "author_keywords",
    "book_title"
]

# Fill missing text
df_clean[text_cols] = df_clean[text_cols].fillna("")

# Fill numbers
df_clean["publication_year"] = df_clean["publication_year"].fillna(-1)

# Remove rows with all empty text
df_clean = df_clean[
    (df_clean["chapter_title"] != "") |
    (df_clean["abstract"] != "") |
    (df_clean["author_keywords"] != "")
]

# Remove duplicates
df_clean = df_clean.drop_duplicates(subset=["chapter_title", "abstract"])

import re

def remove_copyright_year(text):
    if not isinstance(text, str):
        return text
    
    # Remove pattern like: "© 2021", "©2021", "© 1999"
    return re.sub(r'^©\s*\d{4}\s*', '', text)

df_clean["abstract_clean"] = df_clean["abstract"].apply(remove_copyright_year)
df_clean.drop(columns=["abstract"], inplace=True)

df_clean.dropna(inplace=True)

In [41]:
df_clean

,chapter_title,author_keywords,description,book_title,publication_year,ASJC_translation,abstract_clean
1,Flexible Printed Active Antenna for Digital Te...,,This paper presents the development of a flexi...,Progress in Electromagnetics Research Symposium,2018,"Electrical and Electronic Engineering, Materia...","The Institute of Electronics, Information and ..."
2,Parametric study of hydrogen production via so...,Circulating fluidized bed; Computational fluid...,Computational fluid dynamics was applied for s...,Chemical Engineering Science,2018,"Chemistry, Chemical Engineering, Industrial an...",Elsevier LtdComputational fluid dynamics was a...
3,Superhydrophobic coating from fluoroalkylsilan...,Encapsulation; Fluoroalkylsilane; Natural rubb...,A superhydrophobic/superoleophilic mesh was su...,Applied Surface Science,2018,"Chemistry, Condensed Matter Physics, Physics a...",Elsevier B.V. A superhydrophobic/superoleophil...
4,Electrochemical impedance-based DNA sensor usi...,acpcPNA; Electrochemical impedance spectroscop...,A label-free electrochemical DNA sensor based ...,Analytica Chimica Acta,2018,"Analytical Chemistry, Biochemistry, Environmen...",Elsevier B.V. A label-free electrochemical DNA...
5,Evaluation of outsourcing transportation contr...,Design of experiment; Optimal fleet size; Outs...,This paper aims to develop an approach to iden...,Polish Journal of Management Studies,2018,"Unknown(1403), Unknown(1408), Unknown(1407)",", Czestochowa University of Technology. All ri..."
...,...,...,...,...,...,...,...
19800,Long-chain bio-olefins production via oxidativ...,Long-chain olefins; Mesoporous KIT-6; Oleic ac...,Long-chain α-olefins (≥ C10) are normally appl...,Catalysis Today,2023,"Unknown(1503), Chemistry",Elsevier B.V.Long-chain α-olefins (≥ C10) are ...
19801,Recent Developments and Applications of Microf...,Biological hazards; chemical hazards; food con...,"Nowadays, food safety has become a major conce...",Critical Reviews in Analytical Chemistry,2023,Analytical Chemistry,"Taylor & Francis Group, LLC.Nowadays, food saf..."
19802,"Social justice, education and peacebuilding: c...",conflict; Education; peacebuilding; social jus...,Education is increasingly becoming central to ...,Compare,2023,Unknown(3304),The Author(s). Published by Informa UK Limited...
19803,Effects of black soldier fly (Hermetia illucen...,Anabas testudineus; Black soldier fly; fish me...,The effects of replacing fish meal protein wit...,Journal of Applied Aquaculture,2023,"Unknown(2303), Unknown(1104)",Taylor & Francis.The effects of replacing fish...


In [42]:
df_clean.isnull().sum()

chapter_title       0
author_keywords     0
description         0
book_title          0
publication_year    0
ASJC_translation    0
abstract_clean      0
dtype: int64

# Others

In [24]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

class TrendGeneratorAI:
    def __init__(self, dataframe):
        self.df = dataframe.copy()
        # Pre-fill NaNs to prevent errors
        text_cols = ['chapter_title', 'abstract', 'author_keywords', 'ASJC_translation']
        for col in text_cols:
            if col in self.df.columns:
                self.df[col] = self.df[col].fillna('')

    def clean_text(self, text):
        """Basic text cleaning to help the AI understand."""
        if not isinstance(text, str): 
            return ""
        # Lowercase and remove special characters
        text = text.lower()
        text = re.sub(r'[^a-zA-Z\s]', '', text) 
        return text

    def generate_trends(self, category_input, start_year, end_year):
        """
        AI Logic:
        1. Filters data by user input.
        2. Vectorizes text (TF-IDF).
        3. Uses NMF (Generative Topic Modeling) to extract top 3 themes.
        """
        print(f"\n--- 🤖 AI Status: Initializing scan for '{category_input}' ({start_year}-{end_year})... ---")

        # 1. FILTER DATA
        # We use str.contains because a paper might belong to multiple categories
        mask_category = self.df['ASJC_translation'].str.contains(category_input, case=False, na=False)
        mask_year = (self.df['publication_year'] >= start_year) & (self.df['publication_year'] <= end_year)
        
        subset = self.df[mask_category & mask_year].copy()
        
        # Check if we have enough data
        if len(subset) < 5:
            return [f"⚠️ Not enough data ({len(subset)} papers) to generate trends. Try a wider year range."]

        print(f"--- 📚 AI Status: Analyzing {len(subset)} publications... ---")

        # 2. PREPARE TEXT FOR AI
        # We combine Title (x3 weight), Keywords (x2 weight), and Abstract
        # This teaches the AI that Titles are the most important part of the trend.
        subset['ai_text'] = (
            (subset['chapter_title'] + " ") * 3 + 
            (subset['author_keywords'] + " ") * 2 + 
            subset['abstract']
        ).apply(self.clean_text)

        # 3. TF-IDF VECTORIZATION (Math translation of words)
        # ignore words that appear in >95% of docs (too common) or <2 docs (too rare)
        tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
        
        try:
            tfidf = tfidf_vectorizer.fit_transform(subset['ai_text'])
        except ValueError:
            return ["⚠️ Text data is too sparse to generate trends."]

        # 4. NMF MODEL (The 'Generative' Engine)
        # We ask the model to generate exactly 3 latent topics
        n_topics = 3
        nmf_model = NMF(n_components=n_topics, random_state=42, init='nndsvd')
        nmf_model.fit(tfidf)

        # 5. DECODE OUTPUT
        feature_names = tfidf_vectorizer.get_feature_names_out()
        trends = []

        for topic_idx, topic in enumerate(nmf_model.components_):
            # Get the top 4 words that define this trend
            top_indices = topic.argsort()[:-5:-1]
            top_words = [feature_names[i] for i in top_indices]
            
            # Format nicely as a "Title Trend"
            trend_title = " ".join([word.capitalize() for word in top_words])
            trends.append(f"Trend {topic_idx + 1}: {trend_title}")

        return trends

# ==========================================
# 🚀 MAIN EXECUTION BLOCK
# ==========================================

# 1. Load your DataFrame (assuming 'df' is already loaded in your environment)
# If not, uncomment the line below:
# df = pd.read_csv('extracted_publications_with_translation.csv')

# 2. Initialize the AI Engine
ai_engine = TrendGeneratorAI(df)

# 3. USER INPUT SIMULATION
# In a real app, these would come from input boxes
user_category = input("Enter Category (e.g., Medicine, Engineering, Chemistry): ")
try:
    user_start_year = int(input("Enter Start Year (e.g., 2018): "))
    user_end_year = int(input("Enter End Year (e.g., 2021): "))
except ValueError:
    print("Please enter valid numbers for years.")
    user_start_year = 2018
    user_end_year = 2023

# 4. RUN GENERATION
generated_results = ai_engine.generate_trends(user_category, user_start_year, user_end_year)

# 5. DISPLAY RESULTS
print("\n" + "="*40)
print(f"🔥 GENERATED TITLE TRENDS ({user_category})")
print("="*40)
for result in generated_results:
    print(result)
print("="*40)


--- 🤖 AI Status: Initializing scan for 'Medicine' (2016-2018)... ---
--- 📚 AI Status: Analyzing 84 publications... ---

🔥 GENERATED TITLE TRENDS (Medicine)
Trend 1: Adverse Events Incidents Perioperative
Trend 2: Patients Mortality Clinical Study
Trend 3: Pain Version Thai Reliability
